In [1]:
import tensorflow as tf
tf.enable_eager_execution()
import time
import numpy as np
import tempfile
import zipfile
import os
from functools import reduce
from tensorflow_model_optimization.sparsity import keras as sparsity

In [16]:
batch_size = 128
num_classes = 10
shape = (1, 32, 32, 3)

# input image dimensions
img_rows, img_cols, img_dim = 32, 32, 3

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

input_shape = (img_rows, img_cols, img_dim)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
#y_train = tf.keras.utils.to_categorical(y_train, num_classes)
#y_test = tf.keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## Load tflite model

In [10]:
def predict(interpreter, inp, input_index, output_index):
    start = time.time()
    interpreter.set_tensor(input_index, inp)
    interpreter.invoke()
    return time.time() - start, interpreter.get_tensor(output_index)

In [11]:
def average_list(arr):
    return reduce(lambda a, b: a + b, arr) / len(arr)

In [12]:
def eval_model(interpreter, images):
    data = []
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    for img in images:
        inp = img.reshape(shape).astype(np.float32)
        res = predict(interpreter, inp, input_index, output_index)
        data.append(res[0])
    return data

In [35]:
dataset = 'cifar10'
model_name = dataset + '_non-pruned'
tflite_model_file = './models/' + dataset + '/' + model_name + '.tflite'

In [36]:
interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()

In [37]:
eval_model(interpreter, x_test[:1])
results = eval_model(interpreter, x_test)
print('avg_per_pred:', average_list(results))

avg_per_pred: 0.0022827627420425414
